In [1]:
from keras.layers import Input, Dense, LSTM, PReLU, Reshape, Dropout, Activation, BatchNormalization
from keras.models import model_from_json, Model
import keras

from s1p10_model.resnet8_body import resnet8_body

from s1p10_training_utils import save_lstm, DataGenerator, save_layer_weight

import json
import time

import numpy as np

Using TensorFlow backend.


# 0. Pre Training
## 0.1 Load classes' weight & bins edge

In [2]:
with open('./s1p10_data/s1p10_classes_weight.json', 'r') as fp:
    classes_weight = json.load(fp)
    
bins_edge = np.load('./s1p10_data/s1p10_bins_edge.npy')

## 0.2 Global parameter

In [3]:
NUM_CLASSES = len(classes_weight)
NUM_LABELS = 5  # total labels of one training sample
LSTM_NUM_HIDDEN_STATE = 128

## 0.3 Prepare dataset generator

In [4]:
batch_size = 100

gen_param = {'num_class': NUM_CLASSES, 
             'num_labels': NUM_LABELS, 
             'bins_edge': bins_edge,
             'batch_size': batch_size, 
             'shuffle': True, 
             'lstm_dim_hidden_states': LSTM_NUM_HIDDEN_STATE,
             'flip_prob': 0.5}

train_gen = DataGenerator("./s1p10_data/s1p10_CH2_002_output_training.csv", **gen_param)
val_gen = DataGenerator("./s1p10_data/s1p10_CH2_002_output_validation.csv", **gen_param)

# 2. Model Architect

Model comprises of 1 LSTM & 1 Dense classifier. The idea to learn the temporal relation in sequences of steering angle. 

Model inspired by `Improvise Jazz` exercise of DeepLearning.ai

Key point of training this model

\begin{equation}
    x^{<i + 1>}=y^{< i >}
\end{equation}

The output of the previous LSTM block is the input of this LSTM block.

## 2.1 Define share layers

In [5]:
reshapor = Reshape((1, -1))

LSTM_cell = LSTM(LSTM_NUM_HIDDEN_STATE, return_state=True)

densor = Dense(NUM_CLASSES, activation='softmax', 
               name='head')  # use as classifier to output steering angle ID

# 2.2 Define Model

In [6]:
def temporal_model(num_classes, num_labels, lstm_num_hidden):
    """
    Define temporal_model for prediction sequences of steering angle
    
    Input:
        num_classes(int): number of classes of steering angle
        num_prediction(int):
        lstm_num_hidden(int)
    """
    
    # Define input
    X_in_list = [Input(shape=(num_classes, )) for i in range(num_labels - 1)]
    
    # Define initial hidden state for the decoder LSTM
    a_0 = Input(shape=(lstm_num_hidden, ), name="a_0")
    c_0 = Input(shape=(lstm_num_hidden, ), name="c_0")
    
    a = a_0
    c = c_0
    
    outputs = []
    
    for i in range(num_labels - 1):
        # reshape input to be (1, num_classes)
        X = reshapor(X_in_list[i])
        # perform 1 step of LSTM_cell
        a, _, c = LSTM_cell(X, initial_state=[a, c])
        # apply densor
        y = densor(a)
        # store y to outputs
        outputs.append(y)
    
    model = Model(inputs=X_in_list + [a_0, c_0], outputs=outputs)
    return model

In [7]:
model = temporal_model(NUM_CLASSES, NUM_LABELS, LSTM_NUM_HIDDEN_STATE)

model.summary()

model.compile(optimizer='Adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 114)          0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 114)       0           input_1[0][0]                    
                                                                 input_2[0][0]                    
                                                                 input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
a_0 (InputLayer)                (None

# 2. Training

In [14]:
time_str = time.strftime("%Y_%m_%d_%H_%M")

log_dir = './stede_logs/' + time_str
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir,  
                                          batch_size=batch_size, 
                                          update_freq='epoch')

model.fit_generator(train_gen,
                    class_weight=classes_weight,
                    epochs=60,
                    validation_data=val_gen,
                    initial_epoch=40,
                    callbacks=[tb_callback])

Epoch 41/60
182/182 [==============================] - 12s 66ms/step - loss: 3.6972 - head_loss: 0.8705 - head_acc: 0.7025 - head_acc_1: 0.6960 - head_acc_2: 0.7083 - head_acc_3: 0.7152 - val_loss: 3.6789 - val_head_loss: 0.8661 - val_head_acc: 0.7048 - val_head_acc_1: 0.6990 - val_head_acc_2: 0.7099 - val_head_acc_3: 0.7149
Epoch 42/60
182/182 [==============================] - 12s 64ms/step - loss: 3.6948 - head_loss: 0.8711 - head_acc: 0.7040 - head_acc_1: 0.6973 - head_acc_2: 0.7115 - head_acc_3: 0.7141 - val_loss: 3.6741 - val_head_loss: 0.8652 - val_head_acc: 0.6998 - val_head_acc_1: 0.7005 - val_head_acc_2: 0.7125 - val_head_acc_3: 0.7152
Epoch 43/60
182/182 [==============================] - 12s 65ms/step - loss: 3.6816 - head_loss: 0.8695 - head_acc: 0.7051 - head_acc_1: 0.7032 - head_acc_2: 0.7066 - head_acc_3: 0.7120 - val_loss: 3.6557 - val_head_loss: 0.8605 - val_head_acc: 0.7054 - val_head_acc_1: 0.7032 - val_head_acc_2: 0.7130 - val_head_acc_3: 0.7168
Epoch 44/60
182/182

## 2.2 Save weights

In [15]:
# save shared_lstm
save_layer_weight(LSTM_cell, time_str, log_dir, "lstm")

save_layer_weight(densor, time_str, log_dir, "densor")

In [16]:
time_str

'2019_05_28_19_25'